In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt
from itertools import groupby

## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 20

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [3]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


## initial brain and position generator

In [4]:
# initial_dic_of_output_neurons = {}
dic = {}
## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

for individual in individuals_sum_dup_no_self_loop:

    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]
    remove_mid_with_no_predecessor(edges)
    
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))
    
    mid_dic = {}
#     print(edges)
    for item in edges:
        if item[1] in mid_dic:
            mid_dic[item[1]].update({item[0]: item[2]})
        else:
            mid_dic[item[1]] = {item[0]: item[2]}
    
    sum_weights(mid_dic, init_list)
    remove_mid_from_dict(mid_dic)
#     print(mid_dic)    
    dic[individual] = mid_dic
    


In [ ]:
a = [(3, 0),(4,1),(5,2)]

In [ ]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

In [ ]:
a

In [ ]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

In [ ]:
a[-2][0]